In [3]:
# Initial imports.
import pandas as pd
import numpy as np
import datetime as dt
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [4]:
# Loading data after database join of seasons/quarters file and Apple's stock file
# https://towardsdatascience.com/4-tricks-you-should-know-to-parse-date-columns-with-pandas-read-csv-27355bb2ad0e#:~:text=By%20default%2C%20date%20columns%20are%20parsed%20using%20the,a%20different%20date%20format%2C%20for%20example%2C%20YYYY-DD-MM%20HH%3AMM%3ASS%3A
file_path =Path("./AAPL_Mock_ML_Low.csv")
apple_df = pd.read_csv(file_path, parse_dates=['Date'])
apple_df.head()

Date  Season Quarter      Open      High       Low     Close  \
0 1981-02-11  Winter      Q1  0.118304  0.118304  0.117746  0.117746   
1 1981-02-12  Winter      Q1  0.117188  0.117188  0.116629  0.116629   
2 1981-02-13  Winter      Q1  0.114955  0.114955  0.113839  0.113839   
3 1981-02-17  Winter      Q1  0.116629  0.117188  0.116629  0.116629   
4 1981-02-18  Winter      Q1  0.121652  0.122768  0.121652  0.121652   

   Adj Close    Volume  Low_24hr_Change Gain_Loss_Low  
0   0.092428  13843200         0.117746          Gain  
1   0.091552  14560000        -0.001117          Loss  
2   0.089362  11155200        -0.002790          Loss  
3   0.091552  12275200         0.002790          Gain  
4   0.095495  19241600         0.005023          Gain

In [3]:
apple_df['Date'] = pd.to_datetime(apple_df['Date'])
apple_df['Date'] = apple_df['Date'].map(dt.datetime.toordinal)
apple_df.head()

Date  Season Quarter      Open      High       Low     Close  Adj Close  \
0  723222  Winter      Q1  0.118304  0.118304  0.117746  0.117746   0.092428   
1  723223  Winter      Q1  0.117188  0.117188  0.116629  0.116629   0.091552   
2  723224  Winter      Q1  0.114955  0.114955  0.113839  0.113839   0.089362   
3  723228  Winter      Q1  0.116629  0.117188  0.116629  0.116629   0.091552   
4  723229  Winter      Q1  0.121652  0.122768  0.121652  0.121652   0.095495   

     Volume  Low_24hr_Change Gain_Loss_Low  
0  13843200         0.117746          Gain  
1  14560000        -0.001117          Loss  
2  11155200        -0.002790          Loss  
3  12275200         0.002790          Gain  
4  19241600         0.005023          Gain

In [4]:
# Convert date into an integer since the system was not taking it as a float so that the ML model can read the date - for now.
# https://www.geeksforgeeks.org/convert-floats-to-integers-in-a-pandas-dataframe/
#apple_df['Date'] = apple_df['Date'].astype(int)

In [5]:
# Must remain in order to activate label_binarize
#If in doubt refer to sklearn
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
apple_gain_loss_df = apple_df.copy()
apple_gain_loss_df['Gain_Loss_Low'] = le.fit_transform(apple_gain_loss_df['Gain_Loss_Low'])
apple_gain_loss_df.head()

Date  Season Quarter      Open      High       Low     Close  Adj Close  \
0  723222  Winter      Q1  0.118304  0.118304  0.117746  0.117746   0.092428   
1  723223  Winter      Q1  0.117188  0.117188  0.116629  0.116629   0.091552   
2  723224  Winter      Q1  0.114955  0.114955  0.113839  0.113839   0.089362   
3  723228  Winter      Q1  0.116629  0.117188  0.116629  0.116629   0.091552   
4  723229  Winter      Q1  0.121652  0.122768  0.121652  0.121652   0.095495   

     Volume  Low_24hr_Change  Gain_Loss_Low  
0  13843200         0.117746              0  
1  14560000        -0.001117              1  
2  11155200        -0.002790              1  
3  12275200         0.002790              0  
4  19241600         0.005023              0

In [6]:
# If in doubt look up scikit learn label_binarize
from sklearn.preprocessing import label_binarize

binarized_gain_loss = label_binarize(y=list(apple_gain_loss_df['Gain_Loss_Low']), classes=[1,0])
apple_gain_loss_df = apple_df.copy()
apple_gain_loss_df['Gain_Loss_Low'] = binarized_gain_loss
apple_gain_loss_df.head()

Date  Season Quarter      Open      High       Low     Close  Adj Close  \
0  723222  Winter      Q1  0.118304  0.118304  0.117746  0.117746   0.092428   
1  723223  Winter      Q1  0.117188  0.117188  0.116629  0.116629   0.091552   
2  723224  Winter      Q1  0.114955  0.114955  0.113839  0.113839   0.089362   
3  723228  Winter      Q1  0.116629  0.117188  0.116629  0.116629   0.091552   
4  723229  Winter      Q1  0.121652  0.122768  0.121652  0.121652   0.095495   

     Volume  Low_24hr_Change  Gain_Loss_Low  
0  13843200         0.117746              1  
1  14560000        -0.001117              0  
2  11155200        -0.002790              0  
3  12275200         0.002790              1  
4  19241600         0.005023              1

In [7]:
# Perform binary encoding of Season and Quarter columns
apple_binary_encoded = pd.get_dummies(apple_gain_loss_df, columns=["Season", "Quarter"])
apple_binary_encoded.head()

Date      Open      High       Low     Close  Adj Close    Volume  \
0  723222  0.118304  0.118304  0.117746  0.117746   0.092428  13843200   
1  723223  0.117188  0.117188  0.116629  0.116629   0.091552  14560000   
2  723224  0.114955  0.114955  0.113839  0.113839   0.089362  11155200   
3  723228  0.116629  0.117188  0.116629  0.116629   0.091552  12275200   
4  723229  0.121652  0.122768  0.121652  0.121652   0.095495  19241600   

   Low_24hr_Change  Gain_Loss_Low  Season_Spring  Season_Winter  Quarter_Q1  \
0         0.117746              1              0              1           1   
1        -0.001117              0              0              1           1   
2        -0.002790              0              0              1           1   
3         0.002790              1              0              1           1   
4         0.005023              1              0              1           1   

   Quarter_Q2  
0           0  
1           0  
2           0  
3           0  
4           0

In [8]:
# Define features set
X = apple_binary_encoded.copy()
X = X.drop(["Gain_Loss_Low"], axis=1)
X.head()

Date      Open      High       Low     Close  Adj Close    Volume  \
0  723222  0.118304  0.118304  0.117746  0.117746   0.092428  13843200   
1  723223  0.117188  0.117188  0.116629  0.116629   0.091552  14560000   
2  723224  0.114955  0.114955  0.113839  0.113839   0.089362  11155200   
3  723228  0.116629  0.117188  0.116629  0.116629   0.091552  12275200   
4  723229  0.121652  0.122768  0.121652  0.121652   0.095495  19241600   

   Low_24hr_Change  Season_Spring  Season_Winter  Quarter_Q1  Quarter_Q2  
0         0.117746              0              1           1           0  
1        -0.001117              0              1           1           0  
2        -0.002790              0              1           1           0  
3         0.002790              0              1           1           0  
4         0.005023              0              1           1           0

In [9]:
# Define the target set.
y = apple_df["Gain_Loss_Low"].ravel()
y[:5]

array(['Gain', 'Loss', 'Loss', 'Gain', 'Gain'], dtype=object)

In [10]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=100, random_state=78) 

In [13]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [14]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [15]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

Predicted 0  Predicted 1
Actual 0            5            0
Actual 1            1            5

In [16]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [17]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


Predicted 0  Predicted 1
Actual 0            5            0
Actual 1            1            5

Accuracy Score : 0.9090909090909091
Classification Report
              precision    recall  f1-score   support

        Gain       0.83      1.00      0.91         5
        Loss       1.00      0.83      0.91         6

    accuracy                           0.91        11
   macro avg       0.92      0.92      0.91        11
weighted avg       0.92      0.91      0.91        11



In [18]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.05941855, 0.08026179, 0.07167485, 0.09178666, 0.06466225,
       0.0775773 , 0.08421006, 0.44700769, 0.00986184, 0.00696427,
       0.0031209 , 0.00345385])

In [19]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.44700768733017987, 'Low_24hr_Change'),
 (0.09178665790839752, 'Low'),
 (0.08421006166280286, 'Volume'),
 (0.08026179139453313, 'Open'),
 (0.07757729573731513, 'Adj Close'),
 (0.0716748456832342, 'High'),
 (0.06466225382750514, 'Close'),
 (0.05941855254264199, 'Date'),
 (0.009861839123925369, 'Season_Spring'),
 (0.006964271431183198, 'Season_Winter'),
 (0.0034538482350982363, 'Quarter_Q2'),
 (0.003120895123183453, 'Quarter_Q1')]